# Compilation and Speeding Up

Python is an interpreted language.  This means that the code you write is basically being interpreted line by line (this is an oversimplification, but not far from the truth).  Each time a line of code is read, it has to be converted into equivalent machine language instructions.  For example, a `for` loop will need a register to be initialized, an instruction for incrementing the counter, an instruction to check the limits, and suitable branching statements.

When a program is *compiled*, it is converted into machine language once and for all, and only that code is then run.  This also means that any change in the code requires a complete recompilation.  Compared to Python, this is less interactive and takes a longer time to do.

So compiled languages pay a cost at compile time, and reap the benefits at run time.  If you expect that your program is going to run multiple times, then it is usually worth checking if this cost is worth it.

## Speed of Python

Python code is typically slow for a number of reasons:

- Data types are not known ahead of time, and the type of a variable can be dynamically changed.  You can store a string in a variable that previously had an `int` for example, and there will be no conflict.  This makes it hard to optimize variables as you do not know how they will change in future.
- Semantics of certain operations are different in Python than they are in other languages or machine code.  For example, *Divide by Zero* will cause an exception to be raised in Python code.  On the other hand, in C code it will result in the program crashing.  It may be possible to catch this exception in languages like C++, but it is optional and not mandatory, so it is possible to crash as well.  Such checks add extra code and slow the program down.
- Accessing an index that is beyond the bounds of a list will cause an Error to be raised.  In C it will not be an error, but may cause the program to crash with a Segmentation Fault.

Similarly, there are other situations where the semantics of the Python code differ from a similar C or machine language representation.  Whenever this happens, there is a chance that the Python will be slower than the raw code.

## Improving Speed

The simplest approach for speeding things up is to try and convert the Python code to a lower level language like C, compile it, and then run the compiled code.  However, due to the above restrictions, this has to be done with care, to avoid changing the meaning of the program.

## Cython

*Cython* is a particular variant of the Python language: it introduces several new syntactic elements into the language to address the issues with types and compilation.  The usual way of running it is to compile the code into a dynamic library, and then import this into Python.  However, in Jupyter notebooks, there is an easier approach that can be used, which makes use of the Cython extensions and *magic annotations*.

# Timing and Optimization

We first measure the time taken for a simple function.  Then we can look at optimizing this using Cython.

In [ ]:
def isPrime(n):
    for i in range(2,int(n**0.5)+1):
        if n%i==0:
            return False
        
    return True

In [ ]:
%timeit isPrime(999999937)

## Cython

First we just apply cython without any optimizations.  Later we will see the effect of adding the optimizations to it.

In [ ]:
%load_ext Cython

In [ ]:
%%cython --annotate

def cbasic_isPrime(n):
    for i in range(2,int(n**0.5)+1):
        if n%i==0:
            return False
        
    return True

In [ ]:
%timeit cbasic_isPrime(999999937)

### Optimized

Now apply several optimizations

In [ ]:
%%cython --annotate

import cython

@cython.cdivision(True)
def c_isPrime(int n):
    cdef int i
    cdef float sqrtn = (n**0.5)
    cdef int lim = int(sqrtn)+1
    for i in range(2,lim):
        if n%i==0:
            return False
        
    return True

In [ ]:
%timeit c_isPrime(999999937)

# A bigger example

The sum of amicable numbers problem was used in one of the quizzes, and is a somewhat hard problem to solve, as it takes a significant amount of time to run.

In [ ]:
def aliquot(n):
    sum = 0 if n==1 else 1  
    for i in range(2, n // 2 + 1):
        if n % i == 0: 
            sum += i
    return sum 

def amicable(n1, n2):
    s1 = aliquot(n1)
    s2 = aliquot(n2)
    if n1 != n2 and s1 == n2 and s2 == n1:
        return True
    else:
        return False 
    # print(f"aliquot({n}) = {s}")
    
def amsum(N):
    aliq = [0] * (N+1)
    for i in range(2, N+1):
        aliq[i] = aliquot(i)

    sum = 0
    for i in range(2, N+1):
        if aliq[i] <= N and i != aliq[i] and i == aliq[aliq[i]]:
            # print(f"Amicable: {i} and {aliq[i]}")
            sum += i
    return sum 

In [ ]:
%timeit amsum(10000)

In [ ]:
%%cython --annotate

import cython

@cython.cdivision(True)
cpdef c_aliquot(int n):
    cdef int sum
    cdef int i
    sum = 0 if n==1 else 1  
    for i in range(2, n // 2 + 1):
        if n % i == 0: 
            sum += i
    return sum 

def c_amsum(int N):
    # Hack follows since we need to statically allocate - use malloc instead
    cdef int[100000] aliq 
    cdef int i
    cdef int sum
    for i in range(2, N+1):
        aliq[i] = c_aliquot(i)

    sum = 0
    for i in range(2, N+1):
        if aliq[i] <= N and i != aliq[i] and i == aliq[aliq[i]]:
            # print(f"Amicable: {i} and {aliq[i]}")
            sum += i
    return sum 

In [ ]:
%timeit c_amsum(70000)

## Optimisation of Gaussian matrix solver and SPICE solver using Cython

In [1]:
import numpy as np

def eq_solve(coeff_matrix,constant_matrix):
    coeff_matrix=np.array(coeff_matrix,dtype=complex)
    constant_matrix=np.array(constant_matrix,dtype=complex)
    a = "Invalid input!"
    b = "Inconsistent solutions"
    h = len(coeff_matrix)
    l = len(coeff_matrix[0])
    hm = len(constant_matrix)
    if(h!=l | h!=hm):   #making sure that we have a square matrix and that A and B have equal dimensions
        return a
    for i in range(h):
        if(coeff_matrix[i][i]==0):   #checking for 0 diagonal elements
            for i1 in range(i,h):
                if(coeff_matrix[i1][i]!=0):
                    for j1 in range(h):
                        swap = coeff_matrix[i1][j1]
                        coeff_matrix[i1][j1]=coeff_matrix[i][j1]
                        coeff_matrix[i][j1]=swap
                else:
                    return b
        norm = coeff_matrix[i][i]
        for j in range(h):
            coeff_matrix[i][j] /= norm
        constant_matrix[i] /= norm
        for t in range(i+1,h):
            temp=coeff_matrix[t][i]
            for k in range(h):
                coeff_matrix[t][k] -= coeff_matrix[i][k]*temp
            constant_matrix[t] -= temp*constant_matrix[i]
    #At this stage A is an upper triangular matrix
    for i2 in reversed(range(1,h)):  #Converting A to identity matrix
        for t2 in range(i2):
            temp = coeff_matrix[t2][i2]
            coeff_matrix[t2][i2] -= temp * (coeff_matrix[i2][i2])
            constant_matrix[t2] -=  constant_matrix[i2]*temp   
    return constant_matrix
a = np.random.rand(10,10)
b = np.random.rand(10,1)
print(eq_solve(a,b))
%timeit eq_solve(a,b)
c = [list(row) for row in a]
d = [row for row in b]
c = np.array(c,dtype=complex)
d = np.array(d,dtype=complex)

[[-3.943841  +0.j]
 [ 1.02784211-0.j]
 [-0.5832345 +0.j]
 [-1.5927877 +0.j]
 [ 2.7206849 +0.j]
 [ 3.87729726+0.j]
 [ 0.29655712+0.j]
 [ 0.28167638+0.j]
 [-2.93787551+0.j]
 [ 3.97024379-0.j]]
539 µs ± 16 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Here we have the original function from the Week 2 assignment which solves linear equations using Gaussian elimination. The code for this is written in Python language. I haven't made any changes to the original code. 
<br>
As we can see, it takes nearly 500 microseconds on average to solve the linear equation.

In [2]:
%load_ext Cython

In [3]:
%%cython --annotate

import cython
import numpy as np
cimport numpy as fnp

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def ceq_solve(fnp.complex128_t[:,:] coeff_matrix,fnp.complex128_t[:,:] constant_matrix):
    #coeff_matrix=np.array(coeff_matrix)
    #constant_matrix=np.array(constant_matrix)
    cdef char* a = "Invalid input!"
    cdef char* b = "Inconsistent solutions"
    cdef int h = len(coeff_matrix)
    cdef int l =len(coeff_matrix[0])
    cdef int hm = len(constant_matrix)
    cdef int i,j,i1,j1,t,k,i2,t2
    cdef complex norm,swap,temp
    if(h!=l | h!=hm):   #making sure that we have a square matrix and that A and B have equal dimensions
        return a
    for i in range(h):
        if(coeff_matrix[i,i]==0):   #checking for 0 diagonal elements
            for i1 in range(i,h):
                if(coeff_matrix[i1,i]!=0):
                    for j1 in range(h):
                        swap = coeff_matrix[i1,j1]
                        coeff_matrix[i1,j1]=coeff_matrix[i,j1]
                        coeff_matrix[i,j1]=swap
                else:
                    return b
        norm = coeff_matrix[i,i]
        for j in range(h):
            coeff_matrix[i,j] = coeff_matrix[i,j] /norm
        constant_matrix[i][0] = constant_matrix[i][0] /norm
        for t in range(i+1,h):
            temp=coeff_matrix[t,i]
            for k in range(h):
                coeff_matrix[t,k] = coeff_matrix[t,k] - coeff_matrix[i,k]*temp
            constant_matrix[t][0] =constant_matrix[t][0] - temp*constant_matrix[i][0]
    #At this stage A is an upper triangular matrix
    for i2 in reversed(range(1,h)):  #Converting A to identity matrix
        for t2 in range(i2):
            temp = coeff_matrix[t2,i2]
            coeff_matrix[t2,i2] =coeff_matrix[t2,i2] - temp * (coeff_matrix[i2,i2])
            constant_matrix[t2][0] =constant_matrix[t2][0] -  constant_matrix[i2][0]*temp   
    return constant_matrix


In [4]:
%timeit ceq_solve(c,d)
f = ceq_solve(c,d)
f = np.array(f,dtype=complex)
print(f)

1.78 µs ± 18.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
[[-3.943841  +0.j]
 [ 1.02784211+0.j]
 [-0.5832345 +0.j]
 [-1.5927877 +0.j]
 [ 2.7206849 +0.j]
 [ 3.87729726+0.j]
 [ 0.29655712+0.j]
 [ 0.28167638+0.j]
 [-2.93787551+0.j]
 [ 3.97024379-0.j]]


Here I have optimised the above code in cython. The basic code is same as the original but I have made a small change. 
- The code here takes only a numpy array (dtype = complex) as the input
<br>

For optimisation I have first declared all the datatypes at first. In the code the dataypes assigned to a variable are not changing.So declaring the datatype of a variable beforehand optimises the code. I have also used decorators @cython.boundscheck(False) and @cython.wraparound(False), which removes all the runtime checks, further reducing the time. But this might lead to error, if we access array's out of bound element.
The decorator  @cython.cdivision(True) allows us to do C division instead of Python division.
<br>
We can see here it takes 2 microseconds on average to compute the final answer.

### Original code for SPICE solver

In [5]:
import math
def circuit_solve(ckt):
    direct = open(ckt)  #opening the file
    relevant = []
    b= "Invalid input! ( Circuit contains multiple frquencies)"
    dc=0
    ac=0
    flag = False
    for line in direct:  #extracting the circuit info between .circuit and .end
        line = line.strip()
        if(line.startswith('.circuit')):
            flag = True
            continue
        elif(line.startswith('.end')):
            flag = False
        if(flag):
            relevant.append(line)  
    nodes = []
    values = {}
    for line in relevant:
        a = line.split()
        if(len(a)>1):
            for hm in range(2):
                if(a[hm+1] not in nodes):  #creating a list of all nodes
                    nodes.append(a[hm+1])
            if(a[0].startswith('R') | a[0].startswith('L') | a[0].startswith('C')):  #creating a dictionary where key is the element and 
                values[a[0]]=a[3]                                                    # value is the elements value/magnitude
            elif(a[0].startswith('V')): #an independent voltage source will also add a node
                if(a[3] == 'dc'):
                    values[a[0]]=a[4]
                    nodes.append(a[0])
                    dc+=1
                elif(a[3] == 'ac'):
                    values [a[0]] = str(int(a[4])*np.cos(float(a[5])))
                    nodes.append(a[0])
                    ac+=1
            elif(a[0].startswith('I')):
                if(a[3] == 'dc'):
                    values[a[0]]=a[4]
                    dc+=1
                elif(a[3] == 'ac'):
                    values [a[0]] = str(int(a[4])*np.cos(float(a[5])))
                    ac+=1
    n = len(nodes)-1
    if(dc*ac !=0):  #making sure that the circuit has either ac or dc sources, not both(multiple frequencies)
        return b
    frq = []
    freq_check = open(ckt)
    for line in freq_check:
        line = line.strip()
        line = line.split()
        if(len(line)>0):
            if(line[0] == '.ac'):
                if(line[2] not in frq):
                    frq.append(line[2])
    if(len(frq) > 1 ):  #checking for circuits with multiple frequencies
        return b
    if(ac!=0):
        freq = 2*math.pi*float(frq[0])
    circuit_matrix = []
    for i in range(n):
        circuit_matrix.append([0 for j in range(n)])
    const_matrix = [0 for i in range(n)]
    nodes.remove('GND')
    node_dict = {}
    t=0
    for i in nodes:
        node_dict[i] = str(t)
        t+=1 
    try:
        for node in nodes: #performing KCL at each node
            for line in relevant:
                line = line.split()
                if(node in line[1]):
                    if(line[0].startswith('R')): 
                        circuit_matrix[int(node_dict[node])][int(node_dict[node])] += 1/(float(values[line[0]]))
                        if(line[2] != 'GND'):
                            circuit_matrix[int(node_dict[node])][int(node_dict[line[2]])] -= 1/float(values[line[0]])
                    elif(line[0].startswith('I')):
                        const_matrix[int(node_dict[node])] += float(values[line[0]])
                    elif(line[0].startswith('V')):
                        circuit_matrix[int(node_dict[node])][int(node_dict[line[0]])] = -1
                    elif(line[0].startswith('L')):
                        circuit_matrix[int(node_dict[node])][int(node_dict[node])] += 1/complex(0,freq*float(values[line[0]]))
                        if(line[2] != 'GND'):
                            circuit_matrix[int(node_dict[node])][int(node_dict[line[2]])] -= 1/complex(0,freq*float(values[line[0]]))
                    elif(line[0].startswith('C')):
                        circuit_matrix[int(node_dict[node])][int(node_dict[node])] += complex(0,freq*float(values[line[0]]))
                        if(line[2] != 'GND'):
                            circuit_matrix[int(node_dict[node])][int(node_dict[line[2]])] -= complex(0,freq*float(values[line[0]]))
                elif(node in line[2]):
                    if(line[0].startswith('R')):
                        circuit_matrix[int(node_dict[node])][int(node_dict[node])] += 1/(float(values[line[0]]))
                        if(line[1] != 'GND'):
                            circuit_matrix[int(node_dict[node])][int(node_dict[line[1]])] -= 1/float(values[line[0]])
                    elif(line[0].startswith('I')):
                        const_matrix[int(node_dict[node])] -= float(values[line[0]])
                    elif(line[0].startswith('V')):
                        circuit_matrix[int(node_dict[node])][int(node_dict[line[0]])] = 1
                    elif(line[0].startswith('L')):
                        circuit_matrix[int(node_dict[node])][int(node_dict[node])] += 1/complex(0,freq*float(values[line[0]]))
                        if(line[1] != 'GND'):
                            circuit_matrix[int(node_dict[node])][int(node_dict[line[1]])] -= 1/complex(0,freq*float(values[line[0]]))
                    elif(line[0].startswith('C')):
                        circuit_matrix[int(node_dict[node])][int(node_dict[node])] += complex(0,freq*float(values[line[0]]))
                        if(line[1] != 'GND'):
                            circuit_matrix[int(node_dict[node])][int(node_dict[line[1]])] -= complex(0,freq*float(values[line[0]]))
                elif(node == line[0]):
                    if(line[1] != 'GND'):
                        circuit_matrix[int(node_dict[node])][int(node_dict[line[1]])] = 1
                    if(line[2] != 'GND'):
                        circuit_matrix[int(node_dict[node])][int(node_dict[line[2]])] = -1
                    const_matrix[int(node_dict[node])] = float(values[line[0]])
    except:
        print("Cannot analyse capacitors for DC inputs")

    x = eq_solve(circuit_matrix,const_matrix)
    return x


In [6]:
fil = 'ckt4.netlist'
%timeit circuit_solve(fil)
print(circuit_solve(fil))

148 µs ± 4.56 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
[-10.        +0.j   2.22222222+0.j  -5.55555556+0.j  -3.7037037 +0.j]


### Cython optimisation of SPICE solver

Here to optimise the circuit solver code from Week 2 assignment, I have just used the C optimised function for linear equation solver that I made earlier. This is because the circuit_solver code has 3 parts
- Reading the netlist file
- Creating a MNA matrix from the file
- Solving the circuit

The Cython code has greatly optimised the third part as we saw in the lat part. So I compared the timing for the original code and the optimised code.
We can see that there is not much difference in the computation time for these two methods. This is because
- The first two parts are heavily python based.
- Here we only have 5 nodes, the linear equation solver(for 5 variables) is pretty fast even without the optimisation.

>I have used the 'ckt4.netlist' for comparison here

In [7]:
def cbase_circuit_solve(ckt):
    direct = open(str(ckt))  #opening the file
    relevant = []
    b= "Invalid input! ( Circuit contains multiple frquencies)"
    dc=0
    ac=0
    flag = False
    for line in direct:  #extracting the circuit info between .circuit and .end
        line = line.strip()
        if(line.startswith('.circuit')):
            flag = True
            continue
        elif(line.startswith('.end')):
            flag = False
        if(flag):
            relevant.append(line)  
    nodes = []
    values = {}
    for line in relevant:
        a = line.split()
        if(len(a)>1):
            for hm in range(2):
                if(a[hm+1] not in nodes):  #creating a list of all nodes
                    nodes.append(a[hm+1])
            if(a[0].startswith('R') | a[0].startswith('L') | a[0].startswith('C')):  #creating a dictionary where key is the element and 
                values[a[0]]=a[3]                                                    # value is the elements value/magnitude
            elif(a[0].startswith('V')): #an independent voltage source will also add a node
                if(a[3] == 'dc'):
                    values[a[0]]=a[4]
                    nodes.append(a[0])
                    dc+=1
                elif(a[3] == 'ac'):
                    values [a[0]] = str(int(a[4])*np.cos(float(a[5])))
                    nodes.append(a[0])
                    ac+=1
            elif(a[0].startswith('I')):
                if(a[3] == 'dc'):
                    values[a[0]]=a[4]
                    dc+=1
                elif(a[3] == 'ac'):
                    values [a[0]] = str(int(a[4])*np.cos(float(a[5])))
                    ac+=1
    n = len(nodes)-1
    if(dc*ac !=0):  #making sure that the circuit has either ac or dc sources, not both(multiple frequencies)
        return b
    frq = []
    freq_check = open(ckt)
    for line in freq_check:
        line = line.strip()
        line = line.split()
        if(len(line)>0):
            if(line[0] == '.ac'):
                if(line[2] not in frq):
                    frq.append(line[2])
    if(len(frq) > 1 ):  #checking for circuits with multiple frequencies
        return b
    if(ac!=0):
        freq = 2*math.pi*float(frq[0])
    circuit_matrix = []
    for i in range(n):
        circuit_matrix.append([0 for j in range(n)])
    const_matrix = [[0] for i in range(n)]
    nodes.remove('GND')
    node_dict = {}
    t=0
    for i in nodes:
        node_dict[i] = str(t)
        t+=1 
    try:
        for node in nodes: #performing KCL at each node
            for line in relevant:
                line = line.split()
                if(node in line[1]):
                    if(line[0].startswith('R')): 
                        circuit_matrix[int(node_dict[node])][int(node_dict[node])] += 1/(float(values[line[0]]))
                        if(line[2] != 'GND'):
                            circuit_matrix[int(node_dict[node])][int(node_dict[line[2]])] -= 1/float(values[line[0]])
                    elif(line[0].startswith('I')):
                        const_matrix[int(node_dict[node])][0] += float(values[line[0]])
                    elif(line[0].startswith('V')):
                        circuit_matrix[int(node_dict[node])][int(node_dict[line[0]])] = -1
                    elif(line[0].startswith('L')):
                        circuit_matrix[int(node_dict[node])][int(node_dict[node])] += 1/complex(0,freq*float(values[line[0]]))
                        if(line[2] != 'GND'):
                            circuit_matrix[int(node_dict[node])][int(node_dict[line[2]])] -= 1/complex(0,freq*float(values[line[0]]))
                    elif(line[0].startswith('C')):
                        circuit_matrix[int(node_dict[node])][int(node_dict[node])] += complex(0,freq*float(values[line[0]]))
                        if(line[2] != 'GND'):
                            circuit_matrix[int(node_dict[node])][int(node_dict[line[2]])] -= complex(0,freq*float(values[line[0]]))
                elif(node in line[2]):
                    if(line[0].startswith('R')):
                        circuit_matrix[int(node_dict[node])][int(node_dict[node])] += 1/(float(values[line[0]]))
                        if(line[1] != 'GND'):
                            circuit_matrix[int(node_dict[node])][int(node_dict[line[1]])] -= 1/float(values[line[0]])
                    elif(line[0].startswith('I')):
                        const_matrix[int(node_dict[node])][0] -= float(values[line[0]])
                    elif(line[0].startswith('V')):
                        circuit_matrix[int(node_dict[node])][int(node_dict[line[0]])] = 1
                    elif(line[0].startswith('L')):
                        circuit_matrix[int(node_dict[node])][int(node_dict[node])] += 1/complex(0,freq*float(values[line[0]]))
                        if(line[1] != 'GND'):
                            circuit_matrix[int(node_dict[node])][int(node_dict[line[1]])] -= 1/complex(0,freq*float(values[line[0]]))
                    elif(line[0].startswith('C')):
                        circuit_matrix[int(node_dict[node])][int(node_dict[node])] += complex(0,freq*float(values[line[0]]))
                        if(line[1] != 'GND'):
                            circuit_matrix[int(node_dict[node])][int(node_dict[line[1]])] -= complex(0,freq*float(values[line[0]]))
                elif(node == line[0]):
                    if(line[1] != 'GND'):
                        circuit_matrix[int(node_dict[node])][int(node_dict[line[1]])] = 1
                    if(line[2] != 'GND'):
                        circuit_matrix[int(node_dict[node])][int(node_dict[line[2]])] = -1
                    const_matrix[int(node_dict[node])][0] = float(values[line[0]])
    except:
        print("Cannot analyse capacitors for DC inputs")
    circuit_matrix = np.array(circuit_matrix,dtype=complex)
    const_matrix = np.array(const_matrix,dtype =complex)
    x = ceq_solve(circuit_matrix,const_matrix)
    return np.array(x,dtype=complex)

In [8]:
fil2 = "ckt4.netlist"
%timeit cbase_circuit_solve('ckt4.netlist')

131 µs ± 5.45 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
